In [171]:
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [172]:
import sys
import os

sys.path.append(r'C:\Program Files\QGIS 3.20.0\apps\qgis\python') #this is important for loading qgis library
sys.path.append(r'C:\Program Files\QGIS 3.20.0\apps\qgis\python\plugins') #this is important for loading processing library

In [173]:
import qgis

In [174]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

In [175]:
%matplotlib widget

In [176]:
from qgis.gui import *
from qgis.core import *
from qgis.utils import plugins
from PyQt5.QtCore import *
from qgis.analysis import QgsNativeAlgorithms

In [177]:
import pandas as pd
import numpy as np
import pykrige.kriging_tools as kt
from pykrige.ok import OrdinaryKriging
from pykrige.uk import UniversalKriging
import matplotlib.pyplot as plt
import seaborn as sb
import gstools as gs
import geopandas as gpd

In [178]:
from datetime import datetime
import json
from pandas import json_normalize
import requests

In [179]:
from pysal.lib import weights
from pysal.lib import cg as geometry
from pysal.model import spreg
from pysal.explore import esda
from pysal.viz import splot
from splot.esda import plot_moran

import contextily
from shapely.geometry import Polygon,LineString, Point
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics
from sklearn.model_selection import train_test_split

In [180]:
from pulp import *

# Geospatial Data Analysis using Open Source Tools <img src="./Resources/geoworks.png" width="100" align ="right"/> <img src="./Resources/sla.png" width="100" align ="right"/>

Geospatial Data Science is a rapidly growing discipline across a wide range of industries including financial services, real estate, cities and government, management consulting, retail, utilities, telecommunications, and many more. Geospatial data analysis skills have become increasingly valuable for professionals and researchers. Many spatial analysis tools are open sourced, so one can view the source code and contribute without expensive licenses.  

## Data Manipulation and Visualization using PYQGIS & GeoPandas

## Spatial Interpolation using sklearn 

* Spatial interpolation is the procedure of estimating the value of properties at unsampled sites within the area covered by existing observations. 
* Rationale behind spatial interpolation is the observation that points close together in space are more likely to have similar values than points far apart.

### Download Rainfall Data 

In [73]:
# def download_rainfall(dt,days):
#     if dt is not None:
#         datelist = pd.date_range(dt, periods=days).tolist()
#         return datelist
    
# w = interactive(download_rainfall, dt=widgets.DatePicker(),days=widgets.IntSlider(min=1, max=100, step=1, value=30,continuous_update=False))

In [74]:
# display(w)

interactive(children=(DatePicker(value=None, description='dt'), IntSlider(value=30, continuous_update=False, d…

In [76]:
# # download Singapore rainfall data
# responses = [requests.get('https://api.data.gov.sg/v1/environment/rainfall?date='+i.strftime("%Y-%m-%d")) for i in w.result]
# [print(i.status_code) for i in responses]

# for i,response in enumerate(responses):
#     with open(f'Data/Rainfall/test{i}.json', 'w') as f:
#         print(f"download {i}")
#         json.dump(response.json(), f)                

In [49]:
responses_json =[]
for filename in os.listdir('Data/Rainfall'):
    with open(os.path.join('Data/Rainfall', filename), 'r') as f: # open in readonly mode
        response = json.load(f)
        responses_json.append(response)

### Preprocessing Rainfall Data

In [50]:
# df_dict=responses[0].json() # if use online download
df_dict=responses_json[0] # if read files

In [51]:
df_dict['metadata']['stations']

[{'id': 'S77',
  'device_id': 'S77',
  'name': 'Alexandra Road',
  'location': {'latitude': 1.2937, 'longitude': 103.8125}},
 {'id': 'S117',
  'device_id': 'S117',
  'name': 'Banyan Road',
  'location': {'latitude': 1.256, 'longitude': 103.679}},
 {'id': 'S90',
  'device_id': 'S90',
  'name': 'Bukit Timah Road',
  'location': {'latitude': 1.3191, 'longitude': 103.8191}},
 {'id': 'S61',
  'device_id': 'S61',
  'name': 'Chai Chee Street',
  'location': {'latitude': 1.323, 'longitude': 103.9217}},
 {'id': 'S114',
  'device_id': 'S114',
  'name': 'Choa Chu Kang Avenue 4',
  'location': {'latitude': 1.38, 'longitude': 103.73}},
 {'id': 'S11',
  'device_id': 'S11',
  'name': 'Choa Chu Kang Road',
  'location': {'latitude': 1.3746, 'longitude': 103.6938}},
 {'id': 'S50',
  'device_id': 'S50',
  'name': 'Clementi Road',
  'location': {'latitude': 1.3337, 'longitude': 103.7768}},
 {'id': 'S107',
  'device_id': 'S107',
  'name': 'East Coast Parkway',
  'location': {'latitude': 1.3135, 'longitude

In [16]:
df_stations = pd.DataFrame(df_dict['metadata']['stations'])

In [17]:
df_stations

,id,device_id,name,location
0,S77,S77,Alexandra Road,"{'latitude': 1.2937, 'longitude': 103.8125}"
1,S117,S117,Banyan Road,"{'latitude': 1.256, 'longitude': 103.679}"
2,S90,S90,Bukit Timah Road,"{'latitude': 1.3191, 'longitude': 103.8191}"
3,S61,S61,Chai Chee Street,"{'latitude': 1.323, 'longitude': 103.9217}"
4,S114,S114,Choa Chu Kang Avenue 4,"{'latitude': 1.38, 'longitude': 103.73}"
...,...,...,...,...
70,S230,S230,S230,"{'latitude': 1.30167, 'longitude': 103.76444}"
71,S92,S92,South Buona Vista Road,"{'latitude': 1.2841, 'longitude': 103.7886}"
72,S29,S29,Pasir Ris Drive 12,"{'latitude': 1.387, 'longitude': 103.935}"
73,S122,S122,Sembawang Road,"{'latitude': 1.41731, 'longitude': 103.8249}"


In [18]:
df_st= df_stations.drop('location', axis=1).join(pd.DataFrame(df_stations.location.values.tolist()))
df_st

,id,device_id,name,latitude,longitude
0,S77,S77,Alexandra Road,1.29370,103.81250
1,S117,S117,Banyan Road,1.25600,103.67900
2,S90,S90,Bukit Timah Road,1.31910,103.81910
3,S61,S61,Chai Chee Street,1.32300,103.92170
4,S114,S114,Choa Chu Kang Avenue 4,1.38000,103.73000
...,...,...,...,...,...
70,S230,S230,S230,1.30167,103.76444
71,S92,S92,South Buona Vista Road,1.28410,103.78860
72,S29,S29,Pasir Ris Drive 12,1.38700,103.93500
73,S122,S122,Sembawang Road,1.41731,103.82490


In [19]:
# uncomment the following line if you use online requests
# df_rainfall = [pd.DataFrame(response.json()['items']).assign(Day_ID=i+1, Time_ID = pd.DataFrame(response.json()['items']).index+1) for i,response in enumerate(responses)]

# uncomment the next line if you use files in rainfall folder
df_rainfall = [pd.DataFrame(response['items']).assign(Day_ID=i+1, Time_ID = pd.DataFrame(response['items']).index+1) for i,response in enumerate(responses_json)]

In [20]:
df_rainfall_all =pd.concat(df_rainfall, axis=0)

In [21]:
df_rainfall_all

,timestamp,readings,Day_ID,Time_ID
0,2020-12-01T00:05:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...",1,1
1,2020-12-01T00:10:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...",1,2
2,2020-12-01T00:15:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...",1,3
3,2020-12-01T00:20:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...",1,4
4,2020-12-01T00:25:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...",1,5
...,...,...,...,...
282,2020-12-10T23:35:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...",10,283
283,2020-12-10T23:40:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...",10,284
284,2020-12-10T23:45:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...",10,285
285,2020-12-10T23:50:00+08:00,"[{'station_id': 'S77', 'value': 0}, {'station_...",10,286


In [22]:
df = pd.concat([pd.DataFrame(row.readings).assign(Day_ID=row.Day_ID,Time_ID=row.Time_ID) for index, row in df_rainfall_all.iterrows()]).reset_index()
df

,index,station_id,value,Day_ID,Time_ID
0,0,S77,0.0,1,1
1,1,S117,0.0,1,1
2,2,S90,0.0,1,1
3,3,S61,0.0,1,1
4,4,S114,0.0,1,1
...,...,...,...,...,...
211283,68,S08,0.0,10,287
211284,69,S116,0.0,10,287
211285,70,S104,0.0,10,287
211286,71,S100,0.0,10,287


In [23]:
df_agg= df.groupby(['Day_ID','station_id']).sum().drop(['Time_ID','index'],axis=1).reset_index()

In [24]:
df_agg

,Day_ID,station_id,value
0,1,S07,0.0
1,1,S08,0.0
2,1,S100,0.0
3,1,S104,0.0
4,1,S107,0.0
...,...,...,...
747,10,S90,5.6
748,10,S900,43.6
749,10,S91,19.8
750,10,S92,5.6


In [25]:
df_stations

,id,device_id,name,location
0,S77,S77,Alexandra Road,"{'latitude': 1.2937, 'longitude': 103.8125}"
1,S117,S117,Banyan Road,"{'latitude': 1.256, 'longitude': 103.679}"
2,S90,S90,Bukit Timah Road,"{'latitude': 1.3191, 'longitude': 103.8191}"
3,S61,S61,Chai Chee Street,"{'latitude': 1.323, 'longitude': 103.9217}"
4,S114,S114,Choa Chu Kang Avenue 4,"{'latitude': 1.38, 'longitude': 103.73}"
...,...,...,...,...
70,S230,S230,S230,"{'latitude': 1.30167, 'longitude': 103.76444}"
71,S92,S92,South Buona Vista Road,"{'latitude': 1.2841, 'longitude': 103.7886}"
72,S29,S29,Pasir Ris Drive 12,"{'latitude': 1.387, 'longitude': 103.935}"
73,S122,S122,Sembawang Road,"{'latitude': 1.41731, 'longitude': 103.8249}"


In [26]:
df_agg_for_kriging = df_agg.merge(df_st,left_on='station_id',right_on='device_id')

In [27]:
df_agg_for_kriging

,Day_ID,station_id,value,id,device_id,name,latitude,longitude
0,1,S07,0.00000,S07,S07,Lornie Road,1.3415,103.8334
1,2,S07,0.20000,S07,S07,Lornie Road,1.3415,103.8334
2,3,S07,33.00000,S07,S07,Lornie Road,1.3415,103.8334
3,4,S07,21.40000,S07,S07,Lornie Road,1.3415,103.8334
4,5,S07,0.20000,S07,S07,Lornie Road,1.3415,103.8334
...,...,...,...,...,...,...,...,...
736,6,S94,0.00000,S94,S94,Pasir Ris Street 51,1.3662,103.9528
737,7,S94,19.00000,S94,S94,Pasir Ris Street 51,1.3662,103.9528
738,8,S94,0.00000,S94,S94,Pasir Ris Street 51,1.3662,103.9528
739,9,S94,64.19999,S94,S94,Pasir Ris Street 51,1.3662,103.9528


### Visualize Rainfall data

In [28]:
df_rainfall_geoms = gpd.points_from_xy(x=df_agg_for_kriging["longitude"],
                                    y=df_agg_for_kriging["latitude"],
                                    crs="epsg:4326"
                                   )

In [29]:
rainfall = gpd.GeoDataFrame(df_agg_for_kriging,
                                   geometry=df_rainfall_geoms
                                  )

In [30]:
# ensure that the two layers have the same crs
# here we have to transform the crs instead of set a new
rainfall=rainfall.to_crs("epsg:3414")

In [52]:
rainfall=gpd.read_file("Data/rainfall.shp")

In [53]:
sz = gpd.read_file('Data/MP14_Subzone_SE_2017.shp')

In [54]:
@interact(day_id=widgets.IntSlider(min=1, max=10, step=1, value=1,continuous_update=False))
def plot_rainfail_byday(day_id):
    rainfall_day=rainfall.query('Day_ID == ' + str(day_id))
    f, ax = plt.subplots(1, figsize=(9, 4))
    sz.plot(ax=ax, legend=False, facecolor="none",edgecolor="k",lw=0.1)
    rainfall_day.plot(ax=ax, column='value', legend=True, scheme='Quantiles', legend_kwds={'fmt':'{:.0f}'}, cmap='Blues', edgecolor="k")
    ax.set_title('Singapore Rainfall Data: ' + str(day_id))
    plt.axis('equal')
    plt.show()

interactive(children=(IntSlider(value=1, continuous_update=False, description='day_id', max=10, min=1), Output…

### Make a regular grid

We must make a grid, which represents the points we'd like to predict.

In [55]:
extent = min_x, max_x, min_y, max_y = [rainfall.geometry.x.min()-1100, rainfall.geometry.x.max()+10000,
                                       rainfall.geometry.y.min()-6500, rainfall.geometry.y.max()+5200]

In [56]:
gridx, gridy = np.mgrid[min_x:max_x:500, min_y:max_x:500]

In [57]:
plt.figure(figsize=(10,6))
plt.scatter(gridx, gridy, s=10)
ax = plt.gca()
sz.plot(ax=ax, legend=False, facecolor="none",edgecolor="k",lw=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [58]:
gridxy = np.stack([gridx.ravel(), gridy.ravel()]).T
grid_points = gpd.points_from_xy(x=gridxy[:,0], y=gridxy[:,1])
grid_points_df = gpd.GeoDataFrame(geometry=grid_points)
grid_points_within = gpd.sjoin(grid_points_df, sz, how='inner', op='within')

grid_points_within.plot()
ax = plt.gca()
sz.plot(ax=ax, legend=False, facecolor="none",edgecolor="k",lw=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

### Using `scikit-learn.gaussian_process`

Modeling with a Gaussian process is equivalent to kriging. Conveniently, the popular machine learning library `scikit-learn` has a Gaussian process modeling tool.
RBF: Radial-basis function kernel (aka squared-exponential kernel). The RBF kernel is a stationary kernel. It is also known as the “squared exponential” kernel. It is parameterized by a length scale parameter , which can either be a scalar (isotropic variant of the kernel) or a vector with the same number of dimensions as the inputs X (anisotropic variant of the kernel). 

In [59]:
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessRegressor

The main hyperparameters are the kernel, which we just defined, and `alpha`, which controls the smoothness. Larger values imply more noise in the input data, and result in smoother grids; default is very small: 1 &times; 10<sup>-9</sup>.

In [60]:
@interact(day_id=widgets.IntSlider(min=1, max=10, step=1, value=1,continuous_update=False),length_scale_input = widgets.IntSlider(min=500, max=20000, step=500, value=1000,continuous_update=False))
def plot_interpolation_byday(day_id,length_scale_input):
    rainfall_day=rainfall.query('Day_ID == ' + str(day_id))
    points = np.vstack((rainfall_day.geometry.x,rainfall_day.geometry.y)).T
    
    kernel = RBF(length_scale=length_scale_input)
    gp = GaussianProcessRegressor(normalize_y=True,
                                  alpha=0.1,  # Larger values imply more noise in the input data.
                                  kernel=kernel)

    gp.fit(points, rainfall_day.value.values)
    grid_points_pred =gp.predict(gridxy)
    grid_points = gpd.points_from_xy(x=gridxy[:,0], y=gridxy[:,1])
    grid_points_df = gpd.GeoDataFrame(pd.DataFrame(grid_points_pred,columns=['value']), geometry=grid_points)

    grid_points_within = gpd.sjoin(grid_points_df, sz, how='inner', op='within')
    
    f, ax = plt.subplots(1, figsize=(9, 4))
    grid_points_within.plot('value',alpha=0.5,legend=True,ax=ax)
    sz.plot(ax=ax, legend=False, facecolor="none",edgecolor="k",lw=0.1)
    rainfall_day.plot(ax=ax, column='value', legend=True, scheme='Quantiles', legend_kwds={'fmt':'{:.0f}'}, cmap='Blues', edgecolor="k")



interactive(children=(IntSlider(value=1, continuous_update=False, description='day_id', max=10, min=1), IntSli…

## Spatial Regression using PYSAL

Spatial regression is to explicitly introduce space or geographical context into the statistical framework of a regression. 


In the analysis of predictive methods and classifiers, we are interested in analyzing residuals, so called error diagnostics. When the regression model *systematically* mis-predicts some types of observations, geography provides us with an exceptionally-useful tool to show whether or not there are *clusters of error* in our data. 

If we *know* that errors tend to be larger in some areas than in other areas (or if error is "contagious" between observations), then we might be able to exploit this structure to make better predictions.

In [61]:
sz_Y = gpd.read_file('Data/MP14_Subzone_Y_2017.shp')

In [62]:
sz_Y.plot("Y",legend=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [63]:
not_varaible_names=['SUBZONE_N', 'PLN_AREA_N', 'REGION_N','SUBZONE_C', 
                    'PLN_AREA_C', 'REGION_C', 'geometry','stud_lprim',
                    'HDB_AV_1rm','e12k_Over','stud_jcoll','VariableX']
variable_names = sz_Y.columns[~sz_Y.columns.isin(not_varaible_names)]
sz_df = sz_Y[variable_names]

In [64]:
variable_names

Index(['Age_0to6', 'Age_20to29', 'Age_over85', 'HDB_AV_E', 'HDB_AV_2rm',
       'House_AV', 'Condo_AV', 'EC_AV', 'Apart_AV', 'DP', 'FDW', 'WP', 'Y'],
      dtype='object')

### Linear Regression

In [65]:
formula = 'Y ~ Age_0to6'

In [66]:
sz_df = sm.add_constant(sz_df)
results = ols(formula, sz_df).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.625
Model:                            OLS   Adj. R-squared:                  0.623
Method:                 Least Squares   F-statistic:                     530.8
Date:                Wed, 15 Sep 2021   Prob (F-statistic):           7.51e-70
Time:                        21:09:05   Log-Likelihood:                -2960.6
No. Observations:                 321   AIC:                             5925.
Df Residuals:                     319   BIC:                             5933.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1827.5951    161.861     11.291      0.000    1509.145    2146.045
Age_0to6       2.2293      0.097     23.040      0.000       2.039       2.420
==============================================================================
Omnibus:                       43.705   Durbin-Watson:                   1.406
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              269.918
Skew:                           0.285   Prob(JB):                     2.44e-59
Kurtosis:                       7.456   Cond. No.                     1.97e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.97e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [67]:
formula = 'Y ~ Age_0to6 + House_AV + FDW'

In [68]:
sz_df = sm.add_constant(sz_df)
results = ols(formula, sz_df).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.752
Method:                 Least Squares   F-statistic:                     325.0
Date:                Wed, 15 Sep 2021   Prob (F-statistic):           2.33e-96
Time:                        21:09:07   Log-Likelihood:                -2892.4
No. Observations:                 321   AIC:                             5793.
Df Residuals:                     317   BIC:                             5808.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1156.0691    142.857      8.093      0.000     875.002    1437.136
Age_0to6       0.6644      0.149      4.460      0.000       0.371       0.957
House_AV      -0.0035      0.000     -7.592      0.000      -0.004      -0.003
FDW            3.7260      0.305     12.210      0.000       3.126       4.326
==============================================================================
Omnibus:                       11.080   Durbin-Watson:                   1.407
Prob(Omnibus):                  0.004   Jarque-Bera (JB):               19.272
Skew:                           0.169   Prob(JB):                     6.53e-05
Kurtosis:                       4.152   Cond. No.                     5.82e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.82e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [69]:
variable_names_X = variable_names.values.tolist()
variable_names_X.remove('Y')
variable_formula =  'Y ~ ' + "+".join(variable_names_X)
variable_formula

'Y ~ Age_0to6+Age_20to29+Age_over85+HDB_AV_E+HDB_AV_2rm+House_AV+Condo_AV+EC_AV+Apart_AV+DP+FDW+WP'

In [70]:
sz_df = sm.add_constant(sz_df)
results = ols(variable_formula, sz_df).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      Y   R-squared:                       0.852
Model:                            OLS   Adj. R-squared:                  0.847
Method:                 Least Squares   F-statistic:                     148.3
Date:                Wed, 15 Sep 2021   Prob (F-statistic):          3.81e-120
Time:                        21:09:09   Log-Likelihood:                -2810.8
No. Observations:                 321   AIC:                             5648.
Df Residuals:                     308   BIC:                             5697.
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   1154.4332    122.543      9.421      0.000     913.305    1395.561
Age_0to6       0.1034      0.167      0.621      0.535      -0.224       0.431
Age_20to29     0.7890      0.099      7.962      0.000       0.594       0.984
Age_over85     2.2167      0.811      2.734      0.007       0.622       3.812
HDB_AV_E   -5.602e-05      0.002     -0.029      0.977      -0.004       0.004
HDB_AV_2rm    -0.0393      0.040     -0.992      0.322      -0.117       0.039
House_AV      -0.0015      0.000     -3.294      0.001      -0.002      -0.001
Condo_AV      -0.0012      0.001     -0.947      0.344      -0.004       0.001
EC_AV         -0.0050      0.004     -1.233      0.219      -0.013       0.003
Apart_AV   -8.528e-05   6.67e-05     -1.279      0.202      -0.000     4.6e-05
DP            -0.3556      0.261     -1.364      0.173      -0.868       0.157
FDW            2.0297      0.455      4.459      0.000       1.134       2.925
WP             0.0235      0.029      0.814      0.416      -0.033       0.080
==============================================================================
Omnibus:                       92.530   Durbin-Watson:                   1.628
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              220.958
Skew:                           1.400   Prob(JB):                     1.05e-48
Kurtosis:                       5.947   Cond. No.                     2.02e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.02e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [71]:
m_ols = spreg.OLS(
    sz_df[['Y']].values, 
    sz_df[variable_names_X].values,
    name_y='Y', 
    name_x=variable_names_X
)
print(m_ols.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :           Y                Number of Observations:         321
Mean dependent var  :   3805.6199                Number of Variables   :          13
S.D. dependent var  :   4006.4281                Degrees of Freedom    :         308
R-squared           :      0.8524
Adjusted R-squared  :      0.8467
Sum squared residual:757983515.823                F-statistic           :    148.2633
Sigma-square        : 2460985.441                Prob(F-statistic)     :  3.812e-120
S.E. of regression  :    1568.753                Log likelihood        :   -2810.774
Sigma-square ML     : 2361319.364                Akaike info criterion :    5647.547
S.E of regression ML:   1536.6585                Schwarz criterion     :    5696.576

----------------------------------------------------------------------------

In [73]:
sz_Y['Resids']=m_ols.u.flatten()

In [74]:
sz_Y.plot("Resids",legend=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

### Spatial Regression

#### Spatial Autocorrelation

In [75]:
w = weights.contiguity.Queen.from_dataframe(sz_Y)
w.transform = 'R'

('WARNING: ', 26, ' is an island (no neighbors)')
('WARNING: ', 263, ' is an island (no neighbors)')
('WARNING: ', 289, ' is an island (no neighbors)')


In [76]:
w.islands

[26, 263, 289]

In [77]:
knn = weights.KNN.from_dataframe(sz_Y, k=4)

In [78]:
neighbors = w.neighbors.copy()

In [79]:
[neighbors[i].extend(knn.neighbors[i]) for i in w.islands]

[None, None, None]

In [80]:
w_new = weights.W(neighbors)

In [81]:
w_new.transform = 'R'

In [82]:
moran = esda.moran.Moran(sz_Y['Y'], w_new)
plot_moran(moran)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 1000x400 with 2 Axes>,
 array([<AxesSubplot:title={'center':'Reference Distribution'}, xlabel='Moran I: 0.67', ylabel='Density'>,
        <AxesSubplot:title={'center':'Moran Scatterplot (0.67)'}, xlabel='Attribute', ylabel='Spatial Lag'>],
       dtype=object))

In [83]:
moran.p_sim

0.001

#### Spatial Lag

The spatial lag model introduces a spatial lag of the *dependent* variable. In the example we have covered, this would translate into:

$$
{Y_i} = \alpha + \rho {Y_{lag-i}} + \sum_k \beta_k X_{ki} + \epsilon_i
$$

In [84]:
m_GM_Lag = spreg.GM_Lag(
    sz_df[['Y']].values, 
    sz_df[variable_names_X].values,
    w=w_new,
    name_y='Y',
    name_x=variable_names_X
)
print(m_GM_Lag.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :           Y                Number of Observations:         321
Mean dependent var  :   3805.6199                Number of Variables   :          14
S.D. dependent var  :   4006.4281                Degrees of Freedom    :         307
Pseudo R-squared    :      0.9896
Spatial Pseudo R-squared:  0.9759

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT    -505.5846066      40.8002822     -12.3916939       0.0000000
            Age_0to6       0.1956513       0.0434049       4.5075832       0.0000066
          Age_20to29       0.5792295       0.0260074      2

In [85]:
f, ax = plt.subplots(1, figsize=(9, 4))
plt.scatter(x=list(range(0,321)),y=m_ols.u.flatten())
plt.scatter(x=list(range(0,321)),y=m_GM_Lag.u.flatten(),color="grey")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [86]:
sz_Y['Resids2']=m_GM_Lag.u.flatten()

In [87]:
f, (ax1, ax2) = plt.subplots(1,2, figsize=(9, 4))
sz_Y.plot("Resids",legend=True,ax=ax1)
sz_Y.plot("Resids2",legend=True,ax=ax2,cmap='OrRd')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

## Spatial Optimization using PULP

A facility location problem that involves determining the number and location of warehouses that are needed to supply a group of supermarkets. We’ll demonstrate how to construct a mixed-integer programming (MIP) model of this problem, implement this model in the  Python API, and then use the Optimizer to find an optimal solution.

Facility location problems have applications in a wide variety of industries. For example,  this problem can be used to find the optimal location for stores, factories, warehouses for supply chain management and logistics. Other applications range from public policy (e.g. positioning police officers in a city), telecommunications (e.g. cell towers in a network), and determining the locations for natural gas transmission equipment. 

### Problem Description 

A large supermarket chain in the Singapore needs to build warehouses for a set of supermarkets it is opening in Singapore. The locations of the supermarkets have been identified, but the locations of the warehouses have yet to be determined. Several good candidate locations for the warehouses have been identified, but decisions must be made regarding how many warehouses to open and at which candidate locations to build them.

Opening many warehouses would be advantageous as this would reduce the average distance a truck has to drive from the warehouse to the supermarket, and hence reduce the delivery cost. However, opening a warehouse has a fixed cost associated with it. In this example, our goal is to find the optimal tradeoff between delivery costs and the costs of building new facilities.


$i \in I$: Index and set of supermarket locations.

$j \in J$: Index and set of candidate warehouse locations.

$f_{j} \in \mathbb{R}^+$: Fixed cost associated with constructing facility $j \in J$.

$d_{i,j} \in \mathbb{R}^+$: Distance between facility $j \in J$ and customer $i \in I$.

$c_{i,j} \in \mathbb{R}^+$: Cost of shipping between candidate facility site $j \in J$ and customer location $i \in I$. We assume that this cost is proportional to the distance between the facility and the customer. That is, $c_{i,j} = \alpha \cdot d_{i,j}$, where $\alpha$ is the cost per km of driving, adjusted to incorporate the average number of trips a delivery truck would be expected to make over a five year period.

$select_{j} \in \{0, 1 \}$: This variable is equal to 1 if we build a facility at candidate location $j \in J$; and 0 otherwise.

$0 \leq assign_{i,j} \leq 1$: This non-negative continuous variable determines the fraction of supply received by supermarket $i \in I$ from warehouse $j \in J$.

- **Total costs**. We want to minimize the total cost to open and operate the facilities. This is the sum of the cost of opening facilities and the cost related to shipping between facilities and customers. This total cost measures the tradeoff between the cost of building a new facility and the total delivery cost over a five year period.

\begin{equation}
\text{Min} \quad Z = \sum_{j \in J} f_{j} \cdot select_{j} + \sum_{j \in J} \sum_{i \in I} c_{i,j} \cdot assign_{i,j}
\tag{0}
\end{equation}

- **Demand**. For each supermarket  $i \in I$ ensure that its demand is fulfilled. That is, the sum of the fraction received from each facility for each customer must be equal to 1:

\begin{equation}
\sum_{j \in J} assign_{i,j} = 1 \quad \forall i \in I
\tag{1}
\end{equation}

- **Shipping**. We need to ensure that we  only ship from warehouse $j \in J$,  if that facility has actually been built.

\begin{equation}
assign_{i,j} \leq select_{j} \quad \forall i \in I \quad \forall j \in J
\tag{2}
\end{equation}

### Read Shapefiles

In [78]:
Supermarkets_shp = gpd.read_file('Data/Supermarkets.shp')
Warehouses_shp = gpd.read_file('Data/Warehouses.shp')

In [79]:
f, (ax1,ax2) = plt.subplots(1,2, figsize=(12, 4))
sz.plot(ax=ax1, legend=False, facecolor="none",edgecolor="k",lw=0.1)
Supermarkets_shp.plot(ax=ax1, column='Demand', legend=True, scheme='Quantiles', legend_kwds={'fmt':'{:.0f}'}, cmap='Blues', edgecolor="k")
sz.plot(ax=ax2, legend=False, facecolor="none",edgecolor="k",lw=0.1)
Warehouses_shp.plot(ax=ax2, column='Capacity', legend=True, scheme='Quantiles', legend_kwds={'fmt':'{:.0f}'}, edgecolor="k")
plt.axis('equal')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Parameters

In [80]:
# SETS
SUPERMARKETS = list(Supermarkets_shp['Shop_ID'])
WAREHOUSES =  list(Warehouses_shp['WH_ID'])

In [81]:
SUPERMARKETS

[3, 14, 7, 5, 12, 6, 11, 18, 4, 8, 17, 13, 9, 2, 10, 16, 15, 19, 1, 20]

In [82]:
WAREHOUSES

[39,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38]

In [83]:
# PARAMETERS
demand = dict(zip(SUPERMARKETS,Supermarkets_shp['Demand']))
demand

{3: 385,
 14: 782,
 7: 1156,
 5: 1217,
 12: 792,
 6: 630,
 11: 477,
 18: 483,
 4: 1045,
 8: 603,
 17: 643,
 13: 913,
 9: 396,
 2: 693,
 10: 997,
 16: 882,
 15: 1105,
 19: 740,
 1: 384,
 20: 3}

In [84]:
actcost = dict(zip(WAREHOUSES,Warehouses_shp['Fixed_Cost']))
actcost

{39: 23272,
 1: 60975,
 2: 30975,
 3: 58200,
 4: 38200,
 5: 35415,
 6: 44235,
 7: 23292,
 8: 28200,
 9: 32560,
 10: 43357,
 11: 33907,
 12: 34235,
 13: 32982,
 14: 34235,
 15: 32341,
 16: 60975,
 17: 23907,
 18: 53292,
 19: 55883,
 20: 12000,
 21: 28200,
 22: 53907,
 23: 34235,
 24: 28200,
 25: 34235,
 26: 24379,
 27: 54379,
 28: 54379,
 29: 53272,
 30: 44235,
 31: 33272,
 32: 35233,
 33: 33907,
 34: 28200,
 35: 34349,
 36: 31857,
 37: 23292,
 38: 36581}

In [85]:
maxam = dict(zip(WAREHOUSES,Warehouses_shp['Capacity']))
maxam

{39: 2280,
 1: 4104,
 2: 1296,
 3: 3492,
 4: 8112,
 5: 1860,
 6: 2364,
 7: 1344,
 8: 1776,
 9: 1632,
 10: 1620,
 11: 1488,
 12: 2736,
 13: 1056,
 14: 1044,
 15: 1536,
 16: 3204,
 17: 1524,
 18: 2244,
 19: 3744,
 20: 5000,
 21: 3336,
 22: 984,
 23: 1248,
 24: 1920,
 25: 1632,
 26: 3000,
 27: 2352,
 28: 2688,
 29: 1776,
 30: 1536,
 31: 1644,
 32: 3816,
 33: 1008,
 34: 7308,
 35: 1824,
 36: 1392,
 37: 1188,
 38: 8052}

In [90]:
# This function determines the Euclidean distance between a facility and customer sites.
import math
def compute_distance(loc1, loc2):
    dx = loc1[0] - loc2[0]
    dy = loc1[1] - loc2[1]
    return math.sqrt(dx*dx + dy*dy)

In [91]:
SUPERMARKETS_XY = list(zip(Supermarkets_shp.geometry.x,Supermarkets_shp.geometry.y))
WAREHOUSES_XY = list(zip(Warehouses_shp.geometry.x,Warehouses_shp.geometry.y))

In [92]:
cost_per_km = 0.01

In [93]:

transp={}
for i, wh in enumerate(WAREHOUSES):
    dist=[]
    for j, _ in enumerate(SUPERMARKETS):
        dist.append(cost_per_km * compute_distance(WAREHOUSES_XY[i],SUPERMARKETS_XY[j])) # warning: distance is weighted by cost
    transp[wh] = dict(zip(SUPERMARKETS,dist))
transp

{39: {3: 77.61643923394313,
  14: 42.29376352261722,
  7: 33.85798774099471,
  5: 28.88299202163557,
  12: 63.102588551160956,
  6: 144.01599080425896,
  11: 119.76661765210872,
  18: 98.919968886445,
  4: 155.8258331791174,
  8: 93.81108828627235,
  17: 49.62933772089689,
  13: 142.96136538314633,
  9: 120.38390600622624,
  2: 143.3516264734701,
  10: 201.40049687636014,
  16: 95.99390653356501,
  15: 97.80888561537168,
  19: 98.58582284984287,
  1: 126.9125616506722,
  20: 266.15542318882245},
 1: {3: 188.0149032552678,
  14: 152.78501062590124,
  7: 122.69026646801458,
  5: 92.15980678230511,
  12: 62.35302050825839,
  6: 200.6741137421586,
  11: 164.5697297605202,
  18: 42.03546993146379,
  4: 173.90449404045572,
  8: 28.75203600859406,
  17: 97.519839715598,
  13: 132.9459986037315,
  9: 94.24583251139332,
  2: 74.16036458568323,
  10: 222.29720742965506,
  16: 188.2571900643917,
  15: 112.86491149319453,
  19: 63.63465541497613,
  1: 106.20636890984372,
  20: 288.49879690464115},

### Problem Variable

In [94]:
prob = LpProblem("FacilityLocation",LpMinimize)

### Decision Variables 

In [95]:
serv_vars = LpVariable.dicts("Service", [(i,j) for i in SUPERMARKETS for j in WAREHOUSES],0)

In [96]:
use_vars = LpVariable.dicts("UseLocation",WAREHOUSES,lowBound=0, upBound=1, cat=LpBinary)

### Objective Function

In [97]:
prob += lpSum(actcost[j] * use_vars[j] for j in WAREHOUSES) +\
lpSum(transp[j][i] * serv_vars[(i,j)] for j in WAREHOUSES for i in SUPERMARKETS)

### Constraints

In [98]:
for i in SUPERMARKETS:
    prob += lpSum(serv_vars[(i,j)] for j in WAREHOUSES) == demand[i]

In [99]:
for j in WAREHOUSES:
    prob += lpSum(serv_vars[(i,j)] for i in SUPERMARKETS) <=maxam[j] * use_vars[j]

In [100]:
for i in SUPERMARKETS:
    for j in WAREHOUSES:
        prob += serv_vars[(i,j)] <= demand[i] * use_vars[j]

In [101]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

1

In [102]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

Status: Optimal


In [103]:
TOL = 0.0001
results=[]
for i in WAREHOUSES:
    if use_vars[i].varValue > TOL:
        print(f"Establish warehouse at: {i}")
        results.append(i)

Establish warehouse at: 2
Establish warehouse at: 5
Establish warehouse at: 9
Establish warehouse at: 17
Establish warehouse at: 21
Establish warehouse at: 26
Establish warehouse at: 31
Establish warehouse at: 32
Establish warehouse at: 36


In [104]:
results

[2, 5, 9, 17, 21, 26, 31, 32, 36]

In [114]:
print("Therefore, the optimal warehouses consists of\n")
results=[]
for v in prob.variables():
    if v.varValue>0 and v.name.startswith("Service"):
        print(v.name, "=", v.varValue)

Therefore, the optimal warehouses consists of

Service_(1,_5) = 384.0
Service_(10,_32) = 997.0
Service_(11,_9) = 477.0
Service_(12,_17) = 99.0
Service_(12,_2) = 210.0
Service_(12,_21) = 56.0
Service_(12,_31) = 427.0
Service_(13,_21) = 913.0
Service_(14,_17) = 782.0
Service_(15,_26) = 1105.0
Service_(16,_36) = 882.0
Service_(17,_17) = 643.0
Service_(18,_2) = 483.0
Service_(19,_5) = 740.0
Service_(2,_21) = 693.0
Service_(20,_32) = 3.0
Service_(3,_36) = 385.0
Service_(4,_32) = 1045.0
Service_(5,_31) = 1217.0
Service_(6,_9) = 630.0
Service_(7,_26) = 1156.0
Service_(8,_2) = 603.0
Service_(9,_21) = 396.0


### Create LingStrings

In [166]:
import re

customer_id = []
facility_id = []
values = []
for v in prob.variables():
    if v.varValue>0 and v.name.startswith("Service"):
        print(v.name, "=", v.varValue)
        customer_id.append(int(re.findall('\d+', v.name)[0]))
        facility_id.append(int(re.findall('\d+', v.name)[1]))
        values.append(v.varValue)

lines = [LineString([Supermarkets_shp.query(f"Shop_ID == {i}").geometry.values[0], Warehouses_shp.query(f"WH_ID == {j}").geometry.values[0]]) for i,j in zip(customer_id,facility_id)]

result_value = pd.DataFrame({"customer_id":customer_id,"facility_id":facility_id,"value":values})

result_value

# result_lines = gpd.GeoDataFrame(geometry=lines)
# result_lines.plot()

Service_(1,_5) = 384.0
Service_(10,_32) = 997.0
Service_(11,_9) = 477.0
Service_(12,_17) = 99.0
Service_(12,_2) = 210.0
Service_(12,_21) = 56.0
Service_(12,_31) = 427.0
Service_(13,_21) = 913.0
Service_(14,_17) = 782.0
Service_(15,_26) = 1105.0
Service_(16,_36) = 882.0
Service_(17,_17) = 643.0
Service_(18,_2) = 483.0
Service_(19,_5) = 740.0
Service_(2,_21) = 693.0
Service_(20,_32) = 3.0
Service_(3,_36) = 385.0
Service_(4,_32) = 1045.0
Service_(5,_31) = 1217.0
Service_(6,_9) = 630.0
Service_(7,_26) = 1156.0
Service_(8,_2) = 603.0
Service_(9,_21) = 396.0


,customer_id,facility_id,value
0,1,5,384.0
1,10,32,997.0
2,11,9,477.0
3,12,17,99.0
4,12,2,210.0
5,12,21,56.0
6,12,31,427.0
7,13,21,913.0
8,14,17,782.0
9,15,26,1105.0


## Network

In [5]:
!pip show networkx

Name: networkx
Version: 2.6.3
Summary: Python package for creating and manipulating graphs and networks
Home-page: https://networkx.org/
Author: Aric Hagberg
Author-email: hagberg@lanl.gov
License: UNKNOWN
Location: c:\program files\qgis 3.20.0\apps\python39\lib\site-packages
Requires: 
Required-by: spopt, scikit-image, mapclassify


In [14]:
# !pip install -U networkx

In [170]:
import networkx as nx

from networkx.algorithms import approximation as approx

In [18]:
G = nx.Graph()

In [181]:
test = pd.read_csv("Data/test.csv")

In [187]:
test

,origin_id,destination_id,entry_cost,network_cost,exit_cost,total_cost
0,1,2,NaN,6523.746111,NaN,NaN
1,1,3,NaN,5184.269208,NaN,NaN
2,1,4,NaN,8930.640382,NaN,NaN
3,1,5,NaN,7103.512767,NaN,NaN
4,2,1,NaN,6523.746111,NaN,NaN
5,2,3,NaN,6806.746176,NaN,NaN
6,2,4,NaN,3215.268617,NaN,NaN
7,2,5,NaN,5797.580375,NaN,NaN
8,3,1,NaN,5184.269208,NaN,NaN
9,3,2,NaN,6806.746176,NaN,NaN


In [188]:
for index, row in test.iterrows():
    print(row['origin_id'], row['destination_id'])

1.0 2.0
1.0 3.0
1.0 4.0
1.0 5.0
2.0 1.0
2.0 3.0
2.0 4.0
2.0 5.0
3.0 1.0
3.0 2.0
3.0 4.0
3.0 5.0
4.0 1.0
4.0 2.0
4.0 3.0
4.0 5.0
5.0 1.0
5.0 2.0
5.0 3.0
5.0 4.0


In [191]:
[G.add_edge(str(row['origin_id']), str(row['destination_id']), weight=row['network_cost']) for _, row in test.iterrows()]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [193]:
# G.add_edge('A', 'B', weight=4)
# G.add_edge('B', 'D', weight=2)
# G.add_edge('A', 'C', weight=3)
# G.add_edge('C', 'D', weight=4)
nx.shortest_path(G, '1.0', '3.0', weight='weight')

['1.0', '3.0']

In [190]:
G = nx.DiGraph()

In [19]:
G.add_weighted_edges_from({
    ("A", "B", 3), ("A", "C", 17), ("A", "D", 14), ("B", "A", 3),
    ("B", "C", 12), ("B", "D", 16), ("C", "A", 13),("C", "B", 12),
    ("C", "D", 4), ("D", "A", 14), ("D", "B", 15), ("D", "C", 2)
})

In [20]:
from networkx.algorithms import approximation as approx

In [21]:
G = nx.DiGraph()
G.add_weighted_edges_from({
    ("A", "B", 3), ("A", "C", 17), ("A", "D", 14), ("B", "A", 3),
    ("B", "C", 12), ("B", "D", 16), ("C", "A", 13),("C", "B", 12),
    ("C", "D", 4), ("D", "A", 14), ("D", "B", 15), ("D", "C", 2)
})

In [194]:
cycle = approx.simulated_annealing_tsp(G, "greedy", source="1.0")
cost = sum(G[n][nbr]["weight"] for n, nbr in nx.utils.pairwise(cycle))
cycle

['1.0', '3.0', '5.0', '4.0', '2.0', '1.0']

In [212]:
cycle

['1.0', '3.0', '5.0', '4.0', '2.0', '1.0']

In [214]:
[(cycle[i],cycle[i+1],cost) for i,_ in enumerate(cycle) if i<len(cycle)-1]

[('1.0', '3.0', 23700.432468299998),
 ('3.0', '5.0', 23700.432468299998),
 ('5.0', '4.0', 23700.432468299998),
 ('4.0', '2.0', 23700.432468299998),
 ('2.0', '1.0', 23700.432468299998)]

In [195]:
cost

23700.432468299998

In [24]:
incycle = ["D", "B", "A", "C", "D"]
cycle = approx.simulated_annealing_tsp(G, incycle, source="D")
cost = sum(G[n][nbr]["weight"] for n, nbr in nx.utils.pairwise(cycle))
cycle

['D', 'C', 'B', 'A', 'D']

In [25]:
cost

31